In [1]:
# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [5]:
# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'emotion-detection-fer/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [6]:
# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        'emotion-detection-fer/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [7]:
# create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

C:\Users\mitta\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [8]:
# Train the neural network/model
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

C:\Users\mitta\AppData\Local\Temp\ipykernel_12936\3198193097.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


Epoch 1/50
448/448 [==============================] - 146s 322ms/step - loss: 1.7995 - accuracy: 0.2630 - val_loss: 1.7036 - val_accuracy: 0.3327
Epoch 2/50
448/448 [==============================] - 137s 306ms/step - loss: 1.6176 - accuracy: 0.3716 - val_loss: 1.5168 - val_accuracy: 0.4212
Epoch 3/50
448/448 [==============================] - 139s 311ms/step - loss: 1.5072 - accuracy: 0.4174 - val_loss: 1.4365 - val_accuracy: 0.4549
Epoch 4/50
448/448 [==============================] - 138s 307ms/step - loss: 1.4435 - accuracy: 0.4475 - val_loss: 1.4075 - val_accuracy: 0.4681
Epoch 5/50
448/448 [==============================] - 138s 308ms/step - loss: 1.3844 - accuracy: 0.4733 - val_loss: 1.3425 - val_accuracy: 0.4916
Epoch 6/50
448/448 [==============================] - 145s 323ms/step - loss: 1.3343 - accuracy: 0.4931 - val_loss: 1.2996 - val_accuracy: 0.5056
Epoch 7/50
448/448 [==============================] - 147s 329ms/step - loss: 1.2901 - accuracy: 0.5113 - val_loss: 1.2626 -

In [9]:
# save model structure in jason file
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

In [10]:
# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.h5')